In [1]:
import numpy as np

# Fraud Data
fraud_data = np.load('../data/processed/Xf_balanced.npz')
Xf_balanced = fraud_data['arr_0']
yf_balanced = np.load('../data/processed/Xf_balanced.npz')['arr_0']

# Credit Card Data
cc_data = np.load('../data/processed/Xcc_balanced.npz')
Xcc_balanced = cc_data['arr_0']
ycc_balanced = np.load('../data/processed/ycc_balanced.npz')['arr_0']

print(" Loaded balanced datasets for both Fraud and Credit Card.")


 Loaded balanced datasets for both Fraud and Credit Card.


In [2]:
from sklearn.model_selection import train_test_split

# Fraud Dataset
Xf_train, Xf_test, yf_train, yf_test = train_test_split(
    Xf_balanced, yf_balanced, test_size=0.2, random_state=42, stratify=yf_balanced
)

# Credit Card Dataset
Xcc_train, Xcc_test, ycc_train, ycc_test = train_test_split(
    Xcc_balanced, ycc_balanced, test_size=0.3, random_state=42, stratify=ycc_balanced
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier



ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
import pandas as pd

Xf_test_df = pd.DataFrame(Xf_test)
yf_test_df = pd.Series(yf_test, name='class')

Xf_test_df.to_parquet('../data/processed/Xf_test.parquet', index=False)
yf_test_df.to_frame().to_parquet('../data/processed/yf_test.parquet', index=False)

print(" Saved Fraud Test Set for fraud detection model evaluation.")


 Saved Fraud Test Set for fraud detection model evaluation.


In [ ]:
# saved test set for credit card model evaluation
#Xcc_test_df = pd.DataFrame(Xcc_test)
Xcc_test_df = pd.DataFrame(Xcc_test)
ycc_test_df = pd.Series(ycc_test, name='Class')

Xcc_test_df.to_parquet('../data/processed/Xc_test.parquet', index=False)
ycc_test_df.to_frame().to_parquet('../data/processed/yc_test.parquet', index=False)

print(" Saved Credit Card Test Set for credit card model evaluation.")

 

 Saved Credit Card Test Set for credit card model evaluation.


In [ ]:
# Logistic Regression - Fraud
lr_fraud = LogisticRegression(max_iter=1000, random_state=42)
lr_fraud.fit(Xf_train, yf_train)

LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
# Random Forest - Fraud
rf_fraud = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_fraud.fit(Xf_train, yf_train)

RandomForestClassifier(max_depth=10, random_state=42)

| Metric                | Logistic Regression | Random Forest |
| --------------------- | ------------------- | ------------- |
| **Accuracy**          | 80%                 | **87%**       |
| **F1-Score (Fraud)**  | 0.76                | **0.86**      |
| **Recall (Fraud)**    | 0.64                | **0.79**      |
| **Precision (Fraud)** | 0.93                | **0.93**      |
| **ROC AUC**           | 0.87                | **0.94**      |


Key Findings:
Random Forest consistently outperforms Logistic Regression across key metrics.

Recall improved from 64% to 79%, meaning Random Forest detects significantly more fraudulent transactions.

ROC AUC of 0.94 shows strong discriminative ability between fraud and legitimate transactions.

Precision is equally high, meaning both models avoid too many false alarms.

Business Insights:
High Recall is critical in fraud detection — missing a fraudulent transaction may lead to financial loss or regulatory issues.

Random Forest strikes a balance between catching fraud and minimizing false positives (which could annoy genuine customers).

The improved detection rate (79%) with acceptable precision makes Random Forest a strong candidate for operational deployment.

Logistic Regression remains useful as a baseline or fallback model due to its simplicity and interpretability.

In [ ]:
# Logistic Regression - Credit Card
lr_cc = LogisticRegression(max_iter=1000, random_state=42)
lr_cc.fit(Xcc_train, ycc_train)

LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
# Random Forest - Credit Card
rf_cc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_cc.fit(Xcc_train, ycc_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [ ]:
import joblib

# Fraud Models
joblib.dump(lr_fraud, '../models/lr_fraud_model.pkl')
joblib.dump(rf_fraud, '../models/rf_fraud_model.pkl')

# Credit Card Models
joblib.dump(lr_cc, '../models/lr_creditcard_model.pkl')
joblib.dump(rf_cc, '../models/rf_creditcard_model.pkl')

print(" Saved all models.")


 Saved all models.


In [ ]:
# train models logistic regression, random forest, xgboost
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    print(f"--- {model_name} ---")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    y_proba = model.predict_proba(X_test)[:, 1]
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba)}\n")

# Fraud
evaluate_model(lr_fraud, Xf_test, yf_test, "Logistic Regression - Fraud")
evaluate_model(rf_fraud, Xf_test, yf_test, "Random Forest - Fraud")



--- Logistic Regression - Fraud ---
Confusion Matrix:
[[26032  1361]
 [ 9775 17617]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.95      0.82     27393
           1       0.93      0.64      0.76     27392

    accuracy                           0.80     54785
   macro avg       0.83      0.80      0.79     54785
weighted avg       0.83      0.80      0.79     54785

ROC AUC Score: 0.8731452512149225

--- Random Forest - Fraud ---
Confusion Matrix:
[[25727  1666]
 [ 5677 21715]]
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.94      0.88     27393
           1       0.93      0.79      0.86     27392

    accuracy                           0.87     54785
   macro avg       0.87      0.87      0.87     54785
weighted avg       0.87      0.87      0.87     54785

ROC AUC Score: 0.9419867491643783



In [ ]:
# Credit Card model evaluate 
evaluate_model(lr_cc, Xcc_test, ycc_test, "Logistic Regression - Credit Card")
evaluate_model(rf_cc, Xcc_test, ycc_test, "Random Forest - Credit Card")


--- Logistic Regression - Credit Card ---
Confusion Matrix:
[[82729  2247]
 [ 7221 77755]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.97      0.95     84976
           1       0.97      0.92      0.94     84976

    accuracy                           0.94    169952
   macro avg       0.95      0.94      0.94    169952
weighted avg       0.95      0.94      0.94    169952

ROC AUC Score: 0.9889943327635904

--- Random Forest - Credit Card ---
Confusion Matrix:
[[84826   150]
 [ 1676 83300]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     84976
           1       1.00      0.98      0.99     84976

    accuracy                           0.99    169952
   macro avg       0.99      0.99      0.99    169952
weighted avg       0.99      0.99      0.99    169952

ROC AUC Score: 0.9994375325891965



| Model Type          | Dataset          | Model               | Precision (Fraud) | Recall (Fraud) | F1-Score (Fraud) | ROC-AUC |
| ------------------- | ---------------- | ------------------- | ----------------- | -------------- | ---------------- | ------- |
| Logistic Regression | E-Commerce Fraud | Logistic Regression | 0.93              | 0.64           | 0.76             | 0.87    |
| Random Forest       | E-Commerce Fraud | Random Forest       | 0.93              | 0.79           | 0.86             | 0.94    |
| Logistic Regression | Credit Card      | Logistic Regression | 0.97              | 0.92           | 0.95             | 0.99    |
| Random Forest       | Credit Card      | Random Forest       | 1.00              | 0.98           | 0.99             | 1.00    |


# Business Insights
1️#  Model Performance:

The Random Forest model consistently outperforms Logistic Regression on both datasets in terms of recall (which is critical for fraud detection).

For credit card fraud, models achieve near-perfect scores — thanks to the strong signal in PCA-transformed features.

For e-commerce fraud, performance is slightly lower — possibly due to richer and noisier real-world behavioral data.

## Operational Trade-offs:

Credit Card Fraud Models:
High precision and recall mean we can safely deploy with low risk of false positives and high fraud detection, minimizing customer friction.

E-Commerce Fraud Models:
Slightly lower recall means careful tuning is required to avoid blocking legitimate customers.
Might recommend a risk-scoring system for human review rather than fully automated blocking.

## Model Selection Recommendation:

Random Forest (both cases) is the clear choice due to higher recall and AUC.

Logistic Regression can serve as a baseline for monitoring model drift.



## Business Action Points:

 Deploy Random Forest on Credit Card Transactions with confidence.

 For E-Commerce, implement Random Forest with Threshold Tuning or Risk Scoring + Human-in-the-loop Review.

 Monitor false positives rate after d